# OpenAI Function Calling


**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [1]:
import os

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [5]:
from openai import OpenAI

In [6]:
client = OpenAI()
response = client.chat.completions.create(
      model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [7]:
print(response)

ChatCompletion(id='chatcmpl-9dBE12hQLlJPyy62XXxdlXr6ADlLJ', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None))], created=1719124969, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=82, total_tokens=97))


In [8]:
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None)

In [9]:
response_message =response.choices[0].message

In [10]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None)

In [11]:
response_message.content

In [12]:
response_message.function_call

FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather')

In [13]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston'}

In [14]:
args = json.loads(response_message.function_call.arguments)

In [15]:
get_current_weather(args)

'{"location": {"location": "Boston"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [16]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [17]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [18]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response.choices[0].message.content)

Please wait a moment while I check the current weather in Boston.


In [32]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9dBHoehvG6F2DJhSnqX6Lz2ubGD8y', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1719125204, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=83, total_tokens=95))


In [33]:
print(response.choices[0].message.content)

None


- 강제 했기 때문에 할루시네이션이 생김

In [34]:
print(response.choices[0].message.function_call.arguments)

{
  "location": "San Francisco, CA"
}


- functions call 을 하게 되면 아니 경우보다 몇배의 token을 소모 한다.

In [38]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9dBKcaRgxv6AbtK1dEvVns2WMK7O0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1719125378, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=89, total_tokens=99))


In [39]:
print(response.choices[0].message.function_call.arguments)

{
"location": "Boston, MA"
}


In [40]:
messages.append(response.choices[0].message)

In [41]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)
print(observation)

{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


In [42]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [43]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response.choices[0].message.content)

The current weather in Boston, MA is sunny and windy with a temperature of 72°F.
